In [1]:
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation

d:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!pip install keras
import keras

Using TensorFlow backend.


In [4]:
%matplotlib inline
pd.options.mode.chained_assignment = None

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')
submit_y=submit["Survived"]  #最後結果資料

In [6]:
#TRAIN＋TEST資料 
data=train.append(test)
data.reset_index(inplace=True,drop=True)
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [7]:
train['Family_Size']=train['Parch']+train['SibSp']

In [8]:
test['Family_Size']=test['Parch']+test['SibSp']

In [9]:
#補AGE Train , Test
train['Age']=train['Age'].fillna(data['Age'].mean())
test['Age']=test['Age'].fillna(data['Age'].mean())

In [10]:
#補Cabin
train['Cabin']=train['Cabin'].apply(lambda x: str(x)[0] if not pd.isnull(x) else 'NoCabin')
test['Cabin']=test['Cabin'].apply(lambda x: str(x)[0] if not pd.isnull(x) else 'NoCabin')

In [11]:
train['Embarked']=train['Embarked'].fillna('S')

In [12]:
test['Fare']=test['Fare'].fillna(data['Fare'].mean())
train['Ticket_info'] = train['Ticket'].apply(
     lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')
test['Ticket_info'] = test['Ticket'].apply(
     lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')

In [13]:
train['Title1']=train['Name'].str.split(", ",expand=True)[1]
train['Title1']=train['Title1'].str.split(".",expand=True)[0]
train['Title2'] = train['Title1'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','the Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
         ['Miss','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mr','Mr','Mr','Mr','Mr','Mr','Mrs'])
test['Title1']=test['Name'].str.split(", ",expand=True)[1]
test['Title1']=test['Title1'].str.split(".",expand=True)[0]
test['Title2'] = test['Title1'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','the Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
         ['Miss','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mr','Mr','Mr','Mr','Mr','Mr','Mrs'])

In [14]:
#train 轉數字
train['Age'] = train['Age'].astype('category').cat.codes
train['Sex'] = train['Sex'].astype('category').cat.codes
train['Embarked'] = train['Embarked'].astype('category').cat.codes
train['Pclass'] = train['Pclass'].astype('category').cat.codes
train['Cabin'] = train['Cabin'].astype('category').cat.codes
train['Ticket_info'] = train['Ticket_info'].astype('category').cat.codes
train['Fare'] = train['Fare'].astype('category').cat.codes
train['Title1'] = train['Title1'].astype('category').cat.codes
train['Title2'] = train['Title2'].astype('category').cat.codes

In [15]:
# test 轉數字
test['Age'] = test['Age'].astype('category').cat.codes
test['Sex'] = test['Sex'].astype('category').cat.codes
test['Embarked'] = test['Embarked'].astype('category').cat.codes
test['Pclass'] = test['Pclass'].astype('category').cat.codes
test['Cabin'] = test['Cabin'].astype('category').cat.codes
test['Ticket_info'] = test['Ticket_info'].astype('category').cat.codes
test['Fare'] = test['Fare'].astype('category').cat.codes
test['Title1'] = test['Title1'].astype('category').cat.codes
test['Title2'] = test['Title2'].astype('category').cat.codes

In [16]:
titanic_X = pd.DataFrame([
                         train['Sex'],
                         train['Embarked'],
                         train['Pclass'],
                         train['Cabin'],
                         train['Ticket_info']]).T
titanic_y = train["Survived"]

selected_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']
titanic_X=train[selected_features]

In [17]:
test_X = pd.DataFrame([
                         test['Sex'],
                         test['Embarked'],
                         test['Pclass'],
                         test['Cabin'],
                         test['Ticket_info']
                         ]).T

test_y=submit["Survived"]
test_X  = test[selected_features]

In [18]:
#1. 載入模型
from sklearn.svm import SVC
clf = SVC()
clf.fit(titanic_X.values, titanic_y.values)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
test_y_predicted = clf.predict(test_X)

In [20]:
test_y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [21]:
from sklearn import  metrics
accuracy = metrics.accuracy_score( test_y_predicted,test_y)
print(accuracy)

0.710526315789


In [22]:
submit['Survived'] = test_y_predicted

In [23]:
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
PassengerId    418 non-null int64
Survived       418 non-null int64
dtypes: int64(2)
memory usage: 6.6 KB


In [24]:
submit.to_csv('submit.csv', index= False)

In [25]:
#Random Forest
rfc = RandomForestClassifier()

In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(rfc, titanic_X, titanic_y, cv=10).mean()

0.7924038134150494

In [27]:
rfc.fit(titanic_X, titanic_y)
survived_predict = rfc.predict(test_X)

In [28]:
accuracy = metrics.accuracy_score(survived_predict,test_y)
print(accuracy)

0.772727272727


In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(40, input_dim=6, activation=('relu')))
model.add(keras.layers.Dense(30, activation=('relu')))
model.add(keras.layers.Dense(20, activation=('relu')))
model.add(keras.layers.Dense(10, activation=('relu')))
model.add(keras.layers.Dense(1, activation=('sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [30]:
model.fit(titanic_X,titanic_y,validation_split=0.1,epochs=30)

Train on 801 samples, validate on 90 samples
Epoch 1/30
801/801 [==============================] - 1s 751us/step - loss: 1.6209 - val_loss: 0.8353
Epoch 2/30
801/801 [==============================] - 0s 73us/step - loss: 0.7013 - val_loss: 0.6288
Epoch 3/30
801/801 [==============================] - 0s 80us/step - loss: 0.6835 - val_loss: 0.6147
Epoch 4/30
801/801 [==============================] - 0s 80us/step - loss: 0.6285 - val_loss: 0.5723
Epoch 5/30
801/801 [==============================] - 0s 85us/step - loss: 0.6124 - val_loss: 0.5151
Epoch 6/30
801/801 [==============================] - 0s 75us/step - loss: 0.5852 - val_loss: 0.5186
Epoch 7/30
801/801 [==============================] - 0s 80us/step - loss: 0.6042 - val_loss: 0.4797
Epoch 8/30
801/801 [==============================] - 0s 85us/step - loss: 0.5715 - val_loss: 0.4952
Epoch 9/30
801/801 [==============================] - 0s 90us/step - loss: 0.5932 - val_loss: 0.4716
Epoch 10/30
801/801 [========================

In [31]:
y_new = model.predict_classes(test_X)

In [32]:
accuracy = metrics.accuracy_score(survived_predict,y_new)
print(accuracy)

0.813397129187


In [33]:
submit['Survived'] = y_new

In [34]:
submit.to_csv('submit_keras.csv', index= False)